# query 过滤

文本嵌入和向量相似性搜索通过理解文档的含义以及它们之间的相似性来帮助我们查找文档。然而，当需要基于指定范围对信息进行检索时（例如：查找在特定日期创建的所有文档，或者标记在特定类别下的文档），仅仅使用相似性查找的效果并不理想。这就是元数据过滤发挥作用的地方，因为它可以有效地处理这些结构化过滤器，允许用户根据特定属性筛选搜索结果。

在前述第 3 节中我们介绍了如何在构建索引时引入元数据。在本节中，我们主要介绍在完成索引的构建后，如何进一步的通过 query 过滤来提升检索效果。

## 代码实现

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv() 寻找并定位 .env 文件的路径
# load_dotenv() 读取该 .env 文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # OpenAI API key
os.environ['ZHIPUAI_API_KEY'] = os.getenv('ZHIPUAI_API_KEY') # ZHIPUAI API key

os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

In [2]:
from zhipuai_embedding import ZhipuAIEmbeddings

base_embeddings = ZhipuAIEmbeddings(
    api_key = os.environ['ZHIPUAI_API_KEY']
)

In [3]:
import uuid
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

docs = [
    Document(
        page_content="天劫之后，哪吒、敖丙的灵魂虽保住了，但肉身很快会魂飞魄散。太乙真人打算用七色宝莲给二人重塑肉身。但是在重塑肉身的过程中却遇到重重困难，哪吒、敖丙的命运将走向何方？",
        metadata={"title": "哪吒之魔童闹海", "year": 2025, "rating": 8.5, "duration": 144},
    ),
    Document(
        page_content="""姜子牙、姬发带队坚守西岐，家园保卫战一触即发！邓婵玉、闻仲奉商王殷寿之命，率魔家四将等殷商大军征伐西岐，西岐一方得殷郊、雷震子、杨戬、哪吒等相助，更聚全民之力守卫家园。兵戈相对、法术交锋，两大阵营掀起强强对决，关于“封神榜” 的争夺正在继续......""",
        metadata={"title": "封神第二部：战火西岐", "year": 2025, "rating": 6.1, "duration": 144},
    ),
    Document(
        page_content="""熊大、熊二、光头强意外地和来自未来世界的小亮一起穿越到100年后：世界发生巨大灾变，孢子植物全面侵占，人类在末日中艰难求生，整个地球危在旦夕！而这一切的罪魁祸首竟是......光头强？！
    100年前究竟发生了什么？小亮是敌是友？光头强为何背负恶名？人类命运最终会走向何方？熊强三人组能否穿透这层层迷雾，重启未来？""",
        metadata={"title": "熊出没·重启未来", "year": 2025, "rating": 7.1, "minute": 108},
    ),
]
ids = [str(uuid.uuid4()) for _ in docs]
vectorstore = Chroma.from_documents(documents=docs, ids=ids, embedding=base_embeddings)

In [4]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="title",
        description="电影标题",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="电影上映年份",
        type="integer",
    ),
    AttributeInfo(
        name="rating",
        description="电影评分",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="电影评分，范围1-10分", type="float"
    ),
]
document_content_description = "电影简介"
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

In [ ]:
# This example only specifies a filter
retriever.invoke("我想看一部评分超过 8.0 分的电影")